In [13]:
# h horizontal, v vertical
# sums_rows = [23, 14, 15, 13, 25, 4]
# sums_cols = [3, 25, 20, 18, 11, 17]

# table = [
#     [1, 9, 5, 7, 9, 5],
#     [1, 9, 5, 9, 5, 4],
#     [7, 7, 6, 8, 2, 4],
#     [3, 2, 5, 9, 5, 8],
#     [3, 7, 9, 9, 7, 8],
#     [2, 6, 4, 2, 6, 4],
# ]

# sums_rows = [6, 20, 18, 16, 12, 10, 7, 15]
# sums_cols = [12, 1, 12, 23, 14, 1, 18, 23]


def string_to_table(table_string):
    # Split the string into rows based on spaces
    rows = table_string.split()

    # Convert each row into a list of integers (digits)
    table = [[int(char) for char in row] for row in rows]

    return table


def string_to_array(input_string):
    # Split the string by spaces and convert each item to an integer
    array = [int(item) for item in input_string.split()]

    return array


def print_table(table, message):
    print(message)
    for row in table:
        print(row)


def copy_table(input_table):
    new_table = []
    for row in input_table:
        new_table.append(row.copy())
    return new_table


def get_guide(input_table, selection_table):
    guide_table = copy_table(input_table)
    for i in range(table_size):
        for j in range(table_size):
            if selection_table[i][j] == 0:
                guide_table[i][j] = 0
    return guide_table


table_string = "72668952 13565464 95841932 98124529 86252828 84895796 16217546 47256413"
sums_rows_string = "14 8 14 28 10 20 13 9"
sums_cols_string = "38 32 12 2 2 13 6 11"

table = string_to_table(table_string)
unaltered_table = copy_table(table)
print_table(table, "input table")

sums_rows = string_to_array(sums_rows_string)
sums_cols = string_to_array(sums_cols_string)

table_size = len(table)


def generate_select():
    sel_table = []
    for i in range(table_size):
        sel_row = []
        for j in range(table_size):
            sel_row.append(0)
        sel_table.append(sel_row)
    return sel_table


select = generate_select()

print(table_size)
print(table[0][1])  # row, col


def sum_row(my_table, row_idx):
    my_sum = 0
    for i in range(table_size):
        my_sum += my_table[row_idx][i]
    return my_sum


def sum_col(my_table, col_idx):
    my_sum = 0
    for i in range(table_size):
        my_sum += my_table[i][col_idx]
    return my_sum


def get_row(row_idx):
    return table[row_idx][:]


def get_col(col_idx):
    column = [row[col_idx] for row in table]
    return column


def remove_pos(x, y):
    table[x][y] = 0


def select_pos(x, y):
    select[x][y] = 1
    sums_cols[y] -= table[x][y]
    sums_rows[x] -= table[x][y]
    table[x][y] = 0


# remove_pos(2,2)
# select_pos(3,4)


# initial pruning
# for i in range(table_size):  # loop rows
#     for j in range(table_size):  # loop cols
#         if table[i][j] > sums_rows[i] or table[i][j] > sums_cols[j]:
#             remove_pos(i, j)

# # extra sum pruning
# for i in range(table_size):
#     row_sum = sums_rows[i]
#     col_sum = sums_cols[i]

#     row_total_sum = sum_row(table, i)
#     col_total_sum = sum_col(table, i)

#     diff_row = row_total_sum - row_sum
#     diff_col = col_total_sum - col_sum

#     for j in range(table_size):
#         if table[i][j] > diff_row:
#             select_pos(i, j)
#         if table[j][i] > diff_col:
#             select_pos(j, i)


print(sums_rows)
print(sums_cols)

print("Column below")
print(get_col(1))


def find_sum_combs(vector, target_sum):
    result = []

    def backtrack(start, current_combination, current_sum):
        if current_sum == target_sum:
            result.append(list(current_combination))
            return
        if current_sum > target_sum:  # Stop early if the sum exceeds the target
            return

        for i in range(start, len(vector)):
            current_combination.append(vector[i])
            backtrack(
                i + 1, current_combination, current_sum + vector[i]
            )  # Move to the next index
            current_combination.pop()

    backtrack(0, [], 0)
    return result


def list_wo_elem(vector, idx):
    new_vec = []
    for i, elem in enumerate(vector):
        if idx != i:
            new_vec.append(vector[i])
    return new_vec


def sum_vec(vector):
    my_sum = 0
    for elem in vector:
        my_sum += elem
    return my_sum


# Element incompatible on row
def remove_incompatible_elements():
    # Element incompatible on row
    for i in range(table_size):
        for j in range(table_size):
            element = table[i][j]
            # check if element is incompatible on that row
            if sum_row(table, i) == sums_rows[i]:
                continue

            this_row = get_row(i)
            this_row_wo_elem = list_wo_elem(this_row, j)
            remaining_sum = sums_rows[i] - element
            combs = find_sum_combs(this_row_wo_elem, remaining_sum)
            if len(combs) == 0 and element != sums_rows[i]:
                # if element used makes it impossible to get the row sum
                remove_pos(i, j)

    # Element incompatible on col
    for i in range(table_size):
        for j in range(table_size):
            element = table[i][j]
            # check if element is incompatible on that row
            if sum_col(table, j) == sums_cols[j]:
                continue

            this_col = get_col(j)
            this_col_wo_elem = list_wo_elem(this_col, i)
            remaining_sum = sums_cols[j] - element
            combs = find_sum_combs(this_col_wo_elem, remaining_sum)
            if len(combs) == 0 and element != sums_cols[j]:
                # if element used makes it impossible to get the row sum
                remove_pos(i, j)


def select_mandatory_elements():

    # Mandatory element on row
    for i in range(table_size):
        for j in range(table_size):
            element = table[i][j]
            this_row = get_row(i)
            this_row_wo_elem = list_wo_elem(this_row, j)
            combs = find_sum_combs(this_row_wo_elem, sums_rows[i])
            if len(combs) == 0 and element != 0:
                # print("mandatory position", (i, j), "")
                select_pos(i, j)

    # Mandatory element on col
    for i in range(table_size):
        for j in range(table_size):
            element = table[i][j]
            this_col = get_col(j)
            this_col_wo_elem = list_wo_elem(this_col, i)
            combs = find_sum_combs(this_col_wo_elem, sums_cols[j])
            if len(combs) == 0 and element != 0:
                # print("mandatory position", (i, j), "")
                select_pos(i, j)


def are_tables_same(table1, table2):
    for i in range(len(table1)):
        for j in range(len(table1)):
            if table1[i][j] != table2[i][j]:
                return False
    return True


start_token = 1

# print_table(table, "After basic prunign")
old_table = copy_table(table)
# print_table(old_table, "Initial table")

select_mandatory_elements()


while start_token == 1:
    old_table = copy_table(table)
    # print_table(old_table, "Before step")

    remove_incompatible_elements()
    select_mandatory_elements()
    # print_table(table, "After step")

    if are_tables_same(table, old_table):
        # print("true")
        start_token = 0
    # else:
    # print("false")


# print_table(table, "Updated table")
# print_table(table, "Result")
# print_table(select, "selection table")


guide = get_guide(unaltered_table, select)

print_table(guide, "Guide table")

# is there a possiblity of summing the row

input table
[7, 2, 6, 6, 8, 9, 5, 2]
[1, 3, 5, 6, 5, 4, 6, 4]
[9, 5, 8, 4, 1, 9, 3, 2]
[9, 8, 1, 2, 4, 5, 2, 9]
[8, 6, 2, 5, 2, 8, 2, 8]
[8, 4, 8, 9, 5, 7, 9, 6]
[1, 6, 2, 1, 7, 5, 4, 6]
[4, 7, 2, 5, 6, 4, 1, 3]
8
2
[14, 8, 14, 28, 10, 20, 13, 9]
[38, 32, 12, 2, 2, 13, 6, 11]
Column below
[2, 3, 5, 8, 6, 4, 6, 7]
Guide table
[7, 0, 0, 0, 0, 0, 5, 2]
[1, 3, 0, 0, 0, 4, 0, 0]
[9, 5, 0, 0, 0, 0, 0, 0]
[9, 8, 0, 2, 0, 0, 0, 9]
[0, 6, 2, 0, 2, 0, 0, 0]
[8, 4, 8, 0, 0, 0, 0, 0]
[0, 6, 2, 0, 0, 5, 0, 0]
[4, 0, 0, 0, 0, 4, 1, 0]
